<figure>
  <IMG SRC="https://raw.githubusercontent.com/mbakker7/exploratory_computing_with_python/master/tudelft_logo.png" WIDTH=200 ALIGN="right">
</figure>

### ALOS-2 - InSAR datamodel based on sarxarray

In [1]:
import numpy as np
from pathlib import Path
import sarxarray
import matplotlib.pyplot as plt

import xarray as xr
import rioxarray
from scipy.ndimage import uniform_filter
from scipy.spatial import KDTree
from tqdm import tqdm
import time
from datetime import datetime

from matplotlib.dates import DateFormatter, DayLocator
from matplotlib.ticker import MultipleLocator
from skimage.util import view_as_windows

from scipy.linalg import svd
from scipy.linalg import inv
from scipy.linalg import pinv
from scipy.ndimage import generic_filter, label

import cv2 as cv
import matplotlib.colors as colors

from scipy.interpolate import griddata
import re
import os
import bisect

### Specify path of file location

In [2]:
path = Path('data_alos2_seus_p36/')

In [3]:
list_ifgs = [p for p in path.rglob('*cint_srd.raw') if not str(p).endswith('20180117\cint_srd.raw')]
list_ifgs.sort()
list_ifgs

[PosixPath('data_alos2_seus_p36/process_StEustatius_fixed_mtiming_dembased_20180117/20140917/cint_srd.raw'),
 PosixPath('data_alos2_seus_p36/process_StEustatius_fixed_mtiming_dembased_20180117/20150204/cint_srd.raw'),
 PosixPath('data_alos2_seus_p36/process_StEustatius_fixed_mtiming_dembased_20180117/20150916/cint_srd.raw'),
 PosixPath('data_alos2_seus_p36/process_StEustatius_fixed_mtiming_dembased_20180117/20160914/cint_srd.raw'),
 PosixPath('data_alos2_seus_p36/process_StEustatius_fixed_mtiming_dembased_20180117/20170201/cint_srd.raw'),
 PosixPath('data_alos2_seus_p36/process_StEustatius_fixed_mtiming_dembased_20180117/20180117/cint_srd.raw'),
 PosixPath('data_alos2_seus_p36/process_StEustatius_fixed_mtiming_dembased_20180117/20180328/cint_srd.raw'),
 PosixPath('data_alos2_seus_p36/process_StEustatius_fixed_mtiming_dembased_20180117/20181024/cint_srd.raw'),
 PosixPath('data_alos2_seus_p36/process_StEustatius_fixed_mtiming_dembased_20180117/20190102/cint_srd.raw'),
 PosixPath('data_al

In [4]:
#  Create date list to keep track of each radar image

date_list = []
for i in range(len(list_ifgs)):
    prep_date_string = str(list_ifgs[i])
    date = prep_date_string.split('\\')[2]
    date_list.append(date)

date_to_add = '20180117'  # USERCHANGE 2

# Find the index where the new date should be inserted
insert_index = bisect.bisect_left(date_list, date_to_add)

# Insert the new date at the calculated index
date_list.insert(insert_index, date_to_add)
                 
date_list

IndexError: list index out of range

### Metadata
Information about the shape can be extracted from ifgs.res files and are denoted using 'nlines' and 'npixels', respectively.

In [ ]:
# Open the metadata ifgs.res file

filepath = str(path) + '/' + 'process_StEustatius_fixed_mtiming_dembased_20180117/' + date_list[0] + '/ifgs.res'

with open(filepath, 'r') as file:
    content = file.read()
    
# Look through DORIS V5 'ifgs.res' file for shape

lines = r'Number of lines \(multilooked\):\s+(\d+)'
pixels = r'Number of pixels \(multilooked\):\s+(\d+)'
match_lines = re.search(lines, content)
match_pixels = re.search(pixels, content)

if match_lines:
    
    # Extract the number of lines from the matched pattern
    
    num_lines = int(match_lines.group(1))
    print(f"Number of lines: {num_lines}")
else:
    print("Not found in the file.")

if match_pixels:
    
    # Extract the number of pixels from the matched pattern
    
    num_pixels = int(match_pixels.group(1))
    print(f"Number of pixels: {num_pixels}")
else:
    print("Not found in the file.")

In [ ]:
shape=(num_lines, num_pixels)  # obtained from ifgs.res --> nlines= rows ; npixels = columns
dtype = np.dtype([('re', np.float32), ('im', np.float32)])

### Loading the raw interferogram into a `xarray.Dataset`

In [ ]:
# Create xarray.Dataset object from .raw file

ifg_stack = sarxarray.from_binary(list_ifgs, shape, dtype=dtype)

ifg_stack = ifg_stack.chunk({"azimuth":500, "range":500, "time":1 })  # set custom chunk sizes
ifg_stack.complex

In [ ]:
plt.imshow(ifg_stack.phase.isel(time=5))
ifg_stack.phase.isel(time=5).plot(robust=True, cmap='jet')  # cmap='jet'

### Pair Selection

In [ ]:
metadata = open("data_alos2_seus_p36/process_StEustatius_fixed_mtiming_dembased_20180117/baselines_alos2_p36_20180117.txt")   # USERCHANGE 6

metadata = metadata.readlines()
metadata

In [ ]:
B_perp = []
B_date = []
date_list = []

for i in range(len(metadata)):
    split_string = metadata[i].split()
    split_string_subset = metadata[i].split('/')[0]
    
    date_list.append(split_string_subset)
    B_perp.append(float(split_string[3]))
    B_date.append(split_string_subset)
    
format = '%Y%m%d'

B_T = []
for i in B_date:
    formatted = datetime.strptime(i, format)
    B_T.append(formatted.date())
    
# Remove the mother value (BL = 0)
B_perp_original = B_perp.copy()

idx_remove = B_perp.index(0.0)
B_perp.pop(idx_remove)

In [ ]:
# Calculate spatio-temporal baselines for all possible combinations

def B_combinations(B_perp, B_T, date_list, idx_remove):
    
    # Baselines (Bms_perp & Bms_T) for all the unique combinations

    B_perp_combo = []
    B_T_combo = []
    date_list_combo = []
    
    for i in range(len(B_T)):
        for j in range(len(B_T)):
            if(j > i): 
                t_combo = (B_T[i]-B_T[j])
                B_T_combo.append(t_combo.days)
                
                date_combo = (date_list[i] + '-' + date_list[j])
                date_list_combo.append(date_combo)

    for i in range(len(B_perp)):
        for j in range(len(B_perp)):
            if(i > j):
                perp_combo = B_perp[i] - B_perp[j]
                B_perp_combo.append(perp_combo)
            
    # Combine with original list to get all possible combinations (+ original)
    
    B_perp_comp = B_perp_combo.copy()
    B_perp_comp[idx_remove:idx_remove] = B_perp
        
    return B_perp_comp, B_T_combo, date_list_combo, t_combo

In [ ]:
def mod_coh(B_perp, B_T, B_perp_max, B_T_max):
    coh_modelled = max((1 - (np.abs(B_perp)/B_perp_max)),0) * max((1 - (np.abs(B_T)/B_T_max)),0)
    return coh_modelled

# if both negative ; should not be taken into account --> 0 per element;
# for now linear behaviour, could be refined to have quadratic e.g.

In [ ]:
B_perp_comp, B_T_comp,date_list_combo,t_combo = B_combinations(B_perp, B_T, date_list, idx_remove)
len(B_T_comp)

coh_modelled = []
for i in range(len(B_perp_comp)):
    coh = mod_coh(B_perp_comp[i], B_T_comp[i],  14500, 1152)
    coh_modelled.append(coh)  

In [ ]:
# Pair selection algorithm

selection_idx = [i for i, val in enumerate(coh_modelled) if val > 0.3]

pair_selection = []
B_perp_sel = []
B_T_sel = []
for i in selection_idx:
    selection = date_list_combo[i]
    pair_selection.append(selection)
    
len(pair_selection)

In [ ]:
pair_selection

In [ ]:
# Plot baseline configuration PSI

plt.figure(figsize=(20,10))
plt.scatter(B_T,B_perp_original, marker='^',zorder=2, color=['red' if i==0 else 'blue' for i in B_perp_original])

for i in range(len(B_T)):
    if B_perp_original[i] == 0:
        for j in range(len(B_T)):
            if i != j and B_perp_original[j] != 0:
                plt.plot([B_T[i], B_T[j]], [B_perp_original[i], B_perp_original[j]], color='grey', linestyle='-',linewidth=0.5, zorder=1)

# Set the x-axis label format

date_formatter = DateFormatter('%b %Y')  # Format: Month Year
plt.gca().xaxis.set_major_formatter(date_formatter)

# Set the x-axis tick locator

plt.gca().xaxis.set_major_locator(DayLocator(interval=200))

plt.grid()
plt.title('ALOS-2 p36 St.Eustatius - Single-Mother Configuration')
plt.xlabel('Date')
plt.ylabel('Perpendicular Baseline [m]')
# plt.xticks(rotation=90)
plt.show()

In [ ]:
# Plot baseline configuration SBAS

plt.figure(figsize=(20,10))
plt.scatter(B_T,B_perp_original, marker='^',zorder=2, color=['red' if i==0 else 'blue' for i in B_perp_original])

# Connect lines between the pair selection result

start = []
end = []
for pair in pair_selection:
    start_date, end_date = pair.split('-')[0], pair.split('-')[1]
    start_date = datetime.strptime(start_date, '%Y%m%d').date()
    end_date = datetime.strptime(end_date, '%Y%m%d').date()
    
    start.append(start_date)
    end.append(end_date)

for i in range(len(start)):
    idx_i = B_T.index(start[i])
    idx_j = B_T.index(end[i])
    plt.plot([B_T[idx_i], B_T[idx_j]], [B_perp_original[idx_i], B_perp_original[idx_j]], color='grey', linestyle='-', linewidth=0.5, zorder=1)

# Set the x-axis label format

date_formatter = DateFormatter('%b %Y')  # Format: Month Year
plt.gca().xaxis.set_major_formatter(date_formatter)

# Set the x-axis tick locator

plt.gca().xaxis.set_major_locator(DayLocator(interval=365))

plt.grid()
plt.title('ALOS-2 p36 St.Eustatius - Short-Baseline Configuration')
plt.xlabel('Date')
plt.ylabel('Perpendicular Baseline [m]')
plt.show()

In [ ]:
# split_string_subset = pair_selection[0].split('-')[0]
output = []
for i in range(len(pair_selection)):
    split_string0 = pair_selection[i].split('-')[0]
    split_string1 = pair_selection[i].split('-')[1]
    output.append(split_string0)
    output.append(split_string1)

unique_list = list(set(output))
unique_list.sort()

len(unique_list)

In [ ]:
pair_selection_dict = {}

mother_str = '20180117'
mother_idx = unique_list.index(mother_str)

for pair in pair_selection:
    
    # Retrieve the first date from a unique and ordered list and use it as the key
    
    key = pair.split('-')[0]
    index_key = date_list.index(key) ##### !!!!!!!!!!!!!!!!!!!!!!!!!!! CHANGED FROM unique_list.index to this because one data pair not combined in this case
    
    value_toAdd = pair.split('-')[1]
    index_value_toAdd = date_list.index(value_toAdd) ##### !!!!!!!!!!!!!!!!!!!!!!!!!!! CHANGED FROM unique_list.index to this because one data pair not combined in this case
    
    if(index_key in pair_selection_dict):
        combination_list = pair_selection_dict[index_key]
        combination_list.append(index_value_toAdd)
        pair_selection_dict[index_key] = combination_list
    else:
        pair_selection_dict[index_key] = [index_value_toAdd]

pair_selection_dict

### Interferogram Generation

In [ ]:
# Method 2 - Phasor = total radar measurement per pixel

f_mother = 'slave_rsmp.raw'  # Load complex data of mother to obtain amplitude

shape=(num_lines, num_pixels)  # obtained from ifgs.res --> nlines = rows ; npixels = columns
dtype = np.dtype([('re', np.float32), ('im', np.float32)])

mother = [p for p in path.rglob('*slave_rsmp.raw')]
mother = [p for p in mother if '20180117\slave_rsmp.raw' in str(p)]
mother.sort()
mother

mother = sarxarray.from_binary(mother, shape, dtype=dtype)
mother = mother.chunk({"azimuth":200, "range":200, "time":1 })  # set custom chunk sizes

In [ ]:
def calc_angle(x,y,z):
    func = lambda x,y,z: np.angle((y*np.conj(x))/(z)**2)
    pha = xr.apply_ufunc(func, x, y, z, dask='allowed')
    return pha

In [ ]:
def SBAS_structure_m2(i,j):
        
        P0i = phasor.isel(time=i)
        P0j = phasor.isel(time=j)
        
        comp = (P0i*np.conj(P0j))/(mother.amplitude)**2
        amp = np.abs(comp)
        phase = calc_angle(P0j,P0i,mother.amplitude)    
        
        stack_SBAS_combo = xr.DataArray.to_dataset(comp, dim=None, name='complex', promote_attrs=False)
        stack_SBAS_combo['amplitude'] = amp
        stack_SBAS_combo['phase'] = phase
        
        return stack_SBAS_combo 

In [ ]:
# PAIR SELECTION ALGORITHM
phasor = ifg_stack.complex

t_count = 0
coords = []
first_skip = False

i_correction = 0
j_correction = 0

for i in range(len(ifg_stack.time)):
    
    # Check if key = i exists in dict of pairs
    # else skip/continue

    if(i not in pair_selection_dict):
        continue

    # Retrieve the list that are paired with key = i
    pairs = pair_selection_dict[i]
    
    # Correction of the indexes after the mother_idx
    j_correction = 0 # correction for the j has to reset after every i-loop
    if(i > mother_idx):
        i_correction = 1

    for j in pairs:
        
        if(j > mother_idx):
            j_correction = 1
        
        # mother check
        if(i == mother_idx):
            toAdd = ifg_stack.isel(time=j-j_correction)  # CHANGE THIS LINE TO GET CORRECT IFG GET i if mother =j and vice versa.
        elif(j == mother_idx):
            toAdd = ifg_stack.isel(time=i-i_correction)
        else:
            toAdd = SBAS_structure_m2(i-i_correction,j-j_correction)

         # If count is zero, create initial stack_SBAS_combo, else concat to stack_SBAS_combo 
        if(t_count == 0):
            stack_SBAS_combo = toAdd
        else:
            stack_SBAS_combo = xr.concat([stack_SBAS_combo, toAdd], "time")
            
        coords.append(t_count)
        t_count+=1

stack_SBAS_combo = stack_SBAS_combo.assign_coords(time=coords)

stack_SBAS_combo

### Coherence

In [ ]:
# Coherence = magnitude of an ifg pixels/product of the magnitudes of the original image’s pixels.

f_mother = 'slave_rsmp.raw'  # Load complex data of mother to obtain amplitude

shape=(num_lines, num_pixels)  # obtained from ifgs.res --> nlines = rows ; npixels = columns
dtype = np.dtype([('re', np.float32), ('im', np.float32)])

list_daughters = [p for p in path.rglob('*slave_rsmp.raw')]
list_daughters.sort()

# # Path to remove

path_to_mother_rsmp = Path('caroline/Share/projects/antilles/stacks/alos2/alos2_sm3_p36_f340/process_StEustatius_fixed_mtiming_dembased/process_StEustatius_fixed_mtiming_dembased_20180117/20180117/slave_rsmp.raw')

# # Convert path_to_remove to a string

string_to_remove = str(path_to_mother_rsmp)

# # Filter out the path based on the string

list_daughters = [path for path in list_daughters if str(path) != string_to_remove]
list_daughters

daughters_stack = sarxarray.from_binary(list_daughters, shape, dtype=dtype)
daughters_stack = daughters_stack.chunk({"azimuth":200, "range":200, "time":1 })  # set custom chunk sizes

In [ ]:
# Slice the daughters_stack data array into two parts along the t-dim

daughters_stack_before = daughters_stack.isel(time=slice(None, mother_idx))
daughters_stack_after = daughters_stack.isel(time=slice(mother_idx, None))

# Concatenate the two parts with the mother data

daughter_mother_stack_sub = xr.concat([daughters_stack_before, mother],dim="time")
daughter_mother_stack = xr.concat([daughter_mother_stack_sub, daughters_stack_after],dim="time")
daughter_mother_stack = daughter_mother_stack.assign_coords(time=np.arange(0,len(daughters_stack.time)+1,1))
daughter_mother_stack

In [ ]:
# Find zeros in the daughter_mother_stack and alter zero_replacement accordingly

zero_replacement = 0.01

for t in range(len(daughter_mother_stack.time)):
    if (daughter_mother_stack.isel(time=t).amplitude == 0).any(dim=("azimuth", "range")):
        print(f"mother.amplitude contains zero values for time {t}.")
        
        # Get the amplitude values for each t-dim
        amplitude_values = daughter_mother_stack["amplitude"].isel(time=t).values
        
        # Replace zeros with the set replacement value
        amplitude_values[amplitude_values == 0] = zero_replacement
        # print(daughter_mother_stack["amplitude"][t, :, :].shape)
        
        # Update the dataset
        daughter_mother_stack.amplitude.loc[dict(time=t)] = amplitude_values
    else:
        print(f"No zeros found for time {t}")

In [ ]:
def moving_average(data, N):
    
    # Determine the number of rows & columns in the output raster
    
    nrows = shape[0] - N[0] + 1
    ncols = shape[1] - N[1] + 1
    
    # Specify a different window size in x and y direction
    window_size = (22,8)

    # Apply the uniform filter
    filtered_arr = uniform_filter(data, size=window_size)
    
    return filtered_arr, nrows, ncols

In [ ]:
def calc_mags_SLC(SLC_data, windowsize):
        av_mag_SLC = []      
        
        for i in range(len(SLC_data.time)):
            mag_A_i, nrows, ncols = moving_average((SLC_data.amplitude.isel(time=i).values)**2, windowsize)
            av_mag_SLC.append(mag_A_i)
        
        return av_mag_SLC 

In [ ]:
def calc_mav_ifg(ifg_data, windowsize):
        av_I = []      
        
        for i in range(len(ifg_data.time)):
            
            # calculate moving average of the combinations
            mag_I_count, nrows_I, ncols_I = moving_average(ifg_data.complex.isel(time=i).values, windowsize)
            av_I.append(mag_I_count)
        
        return av_I 

In [ ]:
## This code needs to run once

# Calculate mag for all the 11 times
av_mag_SLC = calc_mags_SLC(daughter_mother_stack, np.array([22,8]))

# Calculate mag for all the 32 times
av_I = calc_mav_ifg(stack_SBAS_combo, np.array([22,8]))

# Create initial data variables
av_I_first, nrows_first, ncols_first = moving_average(stack_SBAS_combo.complex.isel(time=0).values, np.array([22,8]))
first_coh = np.abs(av_I_first/(np.sqrt(av_mag_SLC[0]) * np.sqrt(av_mag_SLC[1])))

In [ ]:
# New coherence

def calc_coherence(first_coh, it_length, av_I, av_Ai, nrows, ncols, date_list):

    #################################################################
    # INPUT:
    # first_coh = first coherence I01, to set up xarray datastructure
    # av_I = phasor (ifg.complex) post moving average
    # av_A0 = original amplitude SLC's post moving average
    
    # OUTPUT:
    # coh_combo_stack = coherence ALL possible combo's [numpy array]
    #################################################################
    
    # prepare list with dates; start by filling first one
    
    date_title = []
    
    first_date = date_list[0] + '-' + date_list[1]
    date_title.insert(0, first_date)
    
    # Create data array
    shape_subset = daughter_mother_stack.dims['azimuth'],daughter_mother_stack.dims['range']
    coh_combo_stack = xr.DataArray(first_coh, 
                            coords={'azimuth': np.arange(0,shape_subset[0], 1, dtype=int),
                            'range': np.arange(0, shape_subset[1], 1, dtype=int)}, 
                            dims=["azimuth","range"])
    count = 1
    coords = []
    coords.append(count-1)
    
    first_skipped = False
    
    
#     for i in range(len(pair_selection_dict)):
    for i in pair_selection_dict:
        pairs = pair_selection_dict[i]
        for j in pairs:
            
            # check to skip combinations with the same time and only do unique combinations
            if(j > i):
                   
                # first is already present
                if(first_skipped):
                    
#                     # calculate mag of the combinations
#                     mag_I_count, nrows_I, ncols_I = moving_average(stack_SBAS_combo.amplitude.isel(time=count).values, np.array([20,4]))

                    # calc coh
                    coherence = np.abs(av_I[count])/(np.sqrt(av_Ai[i]) * np.sqrt(av_Ai[j]))
        
                    # prep date titles
                    date = date_list[i] + '-' + date_list[j]
                    date_title.append(date)
                    # date_title = 0
            
                    # add no data values to obtain same size as input shape
                    coh_toAdd = np.zeros((shape_subset[0],shape_subset[1]))
                    coh_toAdd[:] = np.nan
                    coh_toAdd[0:coherence.shape[0], 0:coherence.shape[1]] = coherence
                    
                    # convert to data-array structure
                    coh_toAdd = xr.DataArray(coh_toAdd, 
                            coords={'azimuth': np.arange(0, shape_subset[0], 1, dtype=int),
                            'range': np.arange(0, shape_subset[1], 1, dtype=int)}, 
                            dims=["azimuth","range"])
            
                    # Add coh to the data-array
                    coh_combo_stack = xr.concat([coh_combo_stack, coh_toAdd], dim="time")
                    

                    # Go to the next combination
                    coords.append(count)
                    count+=1    
                    
                # Make true after the first combination: i=0, j=1
                first_skipped = True
    
    coh_combo_stack = coh_combo_stack.astype(np.float32)
    coh_combo_stack = coh_combo_stack.assign_coords(time=coords)
    coh_combo_stack = xr.DataArray.to_dataset(coh_combo_stack, dim=None, name='coherence', promote_attrs=False)
    
    return coh_combo_stack, date_title

In [ ]:
coh_combo_stack, date_title = calc_coherence(first_coh, len(pair_selection_dict), av_I, av_mag_SLC, nrows_first, ncols_first, date_list)

In [ ]:
np.nanmax(coh_combo_stack.coherence.isel(time=0).values)

In [ ]:
max_idx = coh_combo_stack['coherence'].isel(time=0).argmax(['azimuth', 'range'])
int(max_idx['azimuth'].values), int(max_idx['range'].values)

In [ ]:
# first_coh = np.abs(av_I_first/(np.sqrt(av_mag_SLC[0]) * np.sqrt(av_mag_SLC[1])))

np.abs(av_I[0][int(max_idx['azimuth'].values),int(max_idx['range'].values)])/(np.sqrt(av_mag_SLC[0][int(max_idx['azimuth'].values),int(max_idx['range'].values)])*np.sqrt(av_mag_SLC[1][int(max_idx['azimuth'].values),int(max_idx['range'].values)]))

In [ ]:
plt.imshow(coh_combo_stack.coherence.isel(time=1))
coh_combo_stack.coherence.isel(time=1).plot(robust=True, cmap='bone')

In [ ]:
plt.hist(coh_combo_stack.coherence.isel(time=0).values)

### Create interferograms from SLC's

In [ ]:
def calc_angle(x,y):
    func = lambda x,y: np.angle((y*np.conj(x)))
    pha = xr.apply_ufunc(func, x, y, dask='allowed')
    return pha

In [ ]:
P_mother = mother.complex.isel(time=0)

t_count = 0
for i in range(len(daughters_stack.time)):
    
    P_daughter = daughters_stack.complex.isel(time=i)

    comp = P_mother*np.conj(P_daughter)
    amp = np.abs(comp)
    phase = calc_angle(P_daughter,P_mother)
   
    stack_compare = xr.DataArray.to_dataset(comp, dim=None, name='complex', promote_attrs=False)
    stack_compare['amplitude'] = amp
    stack_compare['phase'] = phase
    
    stack_compare.coords['time'] = ('time', [i])
    
    if t_count == 0:
        ifg_stack_compare = stack_compare
    else:
        ifg_stack_compare = xr.concat([ifg_stack_compare, stack_compare], "time")
        
    t_count += 1

In [ ]:
ifg_stack_compare

In [ ]:
# Using cint_srd from Doris

plt.imshow(ifg_stack.amplitude.isel(time=0))
ifg_stack.amplitude.isel(time=0).plot(robust=True, cmap='jet')  # cmap='jet'

In [ ]:
# Using SLC's from Doris

plt.imshow(ifg_stack_compare.amplitude.isel(time=0))
ifg_stack_compare.amplitude.isel(time=0).plot(robust=True, cmap='jet')  # cmap='jet'

In [ ]:
diff_amp = ifg_stack_compare.amplitude.isel(time=0) - ifg_stack.amplitude.isel(time=0)

plt.imshow(diff_amp)
diff_amp.plot(robust=True, cmap='jet')  # cmap='jet
plt.show()

**Repeat steps to calculate new SBAS combinations & corresponding coherence** 

In [ ]:
def calc_angle(x,y,z):
    func = lambda x,y,z: np.angle((y*np.conj(x))/(z)**2)
    pha = xr.apply_ufunc(func, x, y, z, dask='allowed')
    return pha

In [ ]:
# PAIR SELECTION ALGORITHM
phasor = ifg_stack_compare.complex

t_count = 0
coords = []
first_skip = False

i_correction = 0
j_correction = 0

for i in range(len(ifg_stack_compare.time)):
    
    # Check if key = i exists in dict of pairs
    # else skip/continue

    if(i not in pair_selection_dict):
        continue

    # Retrieve the list that are paired with key = i
    pairs = pair_selection_dict[i]
    
    # Correction of the indexes after the mother_idx
    j_correction = 0 # correction for the j has to reset after every i-loop
    if(i > mother_idx):
        i_correction = 1

    for j in pairs:
        
        if(j > mother_idx):
            j_correction = 1
        
        # mother check
        if(i == mother_idx):
            toAdd = ifg_stack_compare.isel(time=j-j_correction)  # CHANGE THIS LINE TO GET CORRECT IFG GET i if mother =j and vice versa.
        elif(j == mother_idx):
            toAdd = ifg_stack_compare.isel(time=i-i_correction)
        else:
            toAdd = SBAS_structure_m2(i-i_correction,j-j_correction)

         # If count is zero, create initial stack_SBAS_combo, else concat to stack_SBAS_combo 
        if(t_count == 0):
            stack_SBAS_combo_compare = toAdd
        else:
            stack_SBAS_combo_compare = xr.concat([stack_SBAS_combo_compare, toAdd], "time")
            
        coords.append(t_count)
        t_count+=1

stack_SBAS_combo_compare = stack_SBAS_combo_compare.assign_coords(time=coords)

stack_SBAS_combo_compare

In [ ]:
## This code needs to run once

# Calculate mag for all the 11 times
av_mag_SLC = calc_mags_SLC(daughter_mother_stack, np.array([22,8]))

# Calculate mag for all the 32 times
av_I = calc_mav_ifg(stack_SBAS_combo_compare, np.array([22,8]))

# Create initial data variables
av_I_first, nrows_first, ncols_first = moving_average(stack_SBAS_combo_compare.complex.isel(time=0).values, np.array([22,8]))
first_coh = np.abs(av_I_first/(np.sqrt(av_mag_SLC[0]) * np.sqrt(av_mag_SLC[1])))

In [ ]:
coh_combo_stack, date_title = calc_coherence(first_coh, len(pair_selection_dict), av_I, av_mag_SLC, nrows_first, ncols_first, date_list)

In [ ]:
np.nanmax(coh_combo_stack.coherence.isel(time=0).values)

In [ ]:
max_idx = coh_combo_stack['coherence'].isel(time=0).argmax(['azimuth', 'range'])
int(max_idx['azimuth'].values), int(max_idx['range'].values)

In [ ]:
# first_coh = np.abs(av_I_first/(np.sqrt(av_mag_SLC[0]) * np.sqrt(av_mag_SLC[1])))

np.abs(av_I[0][int(max_idx['azimuth'].values),int(max_idx['range'].values)])/(np.sqrt(av_mag_SLC[0][int(max_idx['azimuth'].values),int(max_idx['range'].values)])*np.sqrt(av_mag_SLC[1][int(max_idx['azimuth'].values),int(max_idx['range'].values)]))

In [ ]:
plt.imshow(coh_combo_stack.coherence.isel(time=1))
coh_combo_stack.coherence.isel(time=1).plot(robust=True, cmap='bone')

In [ ]:
plt.hist(coh_combo_stack.coherence.isel(time=0).values)